In [2]:
!pip install google-api-python-client

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 144.5 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.0/209.0 kB 44.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.9/96.9 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.9/220.9 kB 51.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.5/181.5 kB 43.3 MB/s eta 0:00:00


In [5]:
!pip install anthropic textblob

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 626.3/626.3 kB 36.5 MB/s eta 0:00:00


In [6]:
import os
import pandas as pd
from googleapiclient.discovery import build
from anthropic import AnthropicBedrock
import random
from textblob import TextBlob
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

pd.set_option('display.max_rows', None)
os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"

# YouTube API 초기화
api_service_name = "youtube"
api_version = "v3"
youtubeAPI_key = 'XXXXXXXXXXXXXXXXXXXXX'     ## API key 변경 필요
youtube = build(api_service_name, api_version, developerKey=youtubeAPI_key)

# AnthropicBedrock 클라이언트 초기화
client = AnthropicBedrock(aws_region="us-west-2")

def analyze_video_content_small(video_id, video_df, comments_df, max_chunk_length=5000):
    # 비디오 정보 가져오기
    video_info = video_df[video_df['VideoID'] == video_id].iloc[0]
    video_title = video_info['Title']
    video_description = video_info['Description']

    # 해당 비디오의 댓글 필터링
    video_comments = comments_df[comments_df['VideoID'] == video_id]

    # 모든 텍스트 결합
    all_text = f"Comments:\n"
    
    for _, comment in video_comments.iterrows():
        all_text += f"Top Level Comment: {comment['topLevelComment']}\n"
        if comment['ThreadComment']:
            all_text += f"Thread Comment: {comment['ThreadComment']}\n"
        all_text += "\n"

    # 텍스트 청크로 나누기
    # chunks = chunk_text(all_text, max_length=max_chunk_length)

    all_analyses = []
    # for chunk in chunks:
    prompt = f"""
    비디오 제목: {video_info['Title']}
    비디오 설명: {video_info['Description']}
    
        다음 YouTube 비디오 콘텐츠(제목, 설명, Top Level Comment, Thread Comment 포함)를 분석해주세요. 당신의 임무는 다음과 같습니다:

        1. 제목, 설명, Top Level Comment, Thread Comment, 좋야요 수를 바탕으로 비디오에서 논의된 주요 주제나 테마를 요약해주세요.
        2. Thread Comment 분석시 Top Level Comment의 내용과 Thread Comment의 Timestamp를 보고 Context를 같이 분석해주세요.
        3. 댓글의 전반적인 감정(긍정적, 부정적, 중립적)을 Context에 맞게 파악해주고 각 감정별로 원문을 정리해주세요.
        4. Thread Comment가 많이 있는 Top Level Comment의 댓글을 분석하고 주된 논의 내용 및 반복적인 키워드를 나열해주고 대표적인 내용의 원문을 보여주세요.
        5. 댓글 작성자들이 반복적으로 제기하는 질문, 우려사항, 관심사를 강조해주고 원문을 보여주세요.
        6. 댓글에서 나타나는 중요한 의견 차이나 논쟁을 파악해주고(좋아요 수를 참고해주세요) 원문을 보여주세요. 
        7. 비디오 제작자에게 제시된 주목할 만한 피드백이나 제안을 확인해주고 원문을 보여주세요.

        비디오 설명이 제공하는 컨텍스트와 댓글 작성자들 간의 상호작용을 고려하여 종합적인 분석을 제공해주세요. 모든 응답은 한국어로 작성해 주세요.
        분석에 근거가 되는 원문을 포함해서 작성해주세요.
        Video Content:
        {all_text}
        """
    print(f"small prompt: {len(prompt)}")
    print(f"small prompt: {len(prompt.encode('utf-8'))}")

    response = client.messages.create(
            model="anthropic.claude-3-5-sonnet-20240620-v1:0",
            max_tokens=4000,
            temperature=0.5,
            messages=[
                {"role": "user", "content": prompt}
            ]
        )
    
    input_tokens = response.usage.input_tokens
    output_tokens = response.usage.output_tokens
    print(f"small 3.5 sonnet Input Tokens: {input_tokens}, Output Tokens: {output_tokens}")


    result = response.content[0].text
    all_analyses.append(result)

    return " ".join(all_analyses)

def get_top_comments(comments, n=100):
    return sorted(comments, key=lambda x: x['likeCount'], reverse=True)[:n]

def analyze_sentiment(text):
    return TextBlob(text).sentiment.polarity

def cluster_comments(comments, n_clusters=5):
    vectorizer = TfidfVectorizer(max_features=1000, stop_words='english')
    X = vectorizer.fit_transform([c['topLevelComment'] for c in comments])
    kmeans = KMeans(n_clusters=n_clusters)
    kmeans.fit(X)
    return kmeans.labels_

def analyze_video_content_large(video_id, video_df, comments_df, max_chunk_length=5000):
    video_info = video_df[video_df['VideoID'] == video_id].iloc[0]
    video_comments = comments_df[comments_df['VideoID'] == video_id]
    
    # 인기 댓글 분석
    top_comments = get_top_comments(video_comments.to_dict('records'), n=100)
    
    # 감정 분석
    sentiments = [analyze_sentiment(c['topLevelComment']) for c in top_comments]
    avg_sentiment = sum(sentiments) / len(sentiments)
    
    # 클러스터링
    cluster_labels = cluster_comments(video_comments.to_dict('records'))
    
    # Thread가 많은 댓글 필터링
    comments_with_threads = [c for c in video_comments.to_dict('records') if c['ThreadComment']]
    top_thread_comments = sorted(comments_with_threads, key=lambda x: len(x['ThreadComment']), reverse=True)[:10]
    
    
    # Thread가 많은 댓글 그룹핑 및 분석
    thread_comment_analysis = ""
    input_tokens_result =0
    output_tokens_result =0
    for c in top_thread_comments:
        top_comment = c['topLevelComment']
        thread_comments = c['ThreadComment']
        
        # 각 그룹에 대해 감정 분석, 주요 주제 및 키워드 분석
        all_comments_text = f"Top Level Comment: {top_comment}\n" + "\n".join(thread_comments)
        thread_sentiment = analyze_sentiment(all_comments_text)
        
        # 대표적인 댓글과 주요 주제 추출
        prompt= f"""Top Level Comment와 Thread Comments를 포함한 다음 댓글 그룹을 분석해주세요:Top Level Comment: {top_comment} 
        Thread Comments: {''.join([f'- {t}, ' for t in thread_comments])} 
        
        1. 분석에 근거가 되는 원문을 포함해서 이 그룹에서 논의된 주요 주제나 테마를 요약해주세요.
        2. 분석에 근거가 되는 원문을 포함해서 전체적인 감정(긍정적, 부정적, 중립적)을 파악해주세요.
        3. 분석에 근거가 되는 원문을 포함해서 반복적으로 나타나는 키워드나 표현을 요약해주세요.
        4. 분석에 근거가 되는 원문을 포함해서 중요한 논쟁이나 의견 차이가 있는 경우 이를 식별하고 설명해주세요.
        5. 분석에 근거가 되는 원문을 포함해서 주요 논의 내용 및 전반적인 톤을 요약해주세요.
        6. 분석근거의 원문을 생략없이 최대한 많이 포함해서 보여주세요.
        """
        
        print(f"haiku len prompt: {len(prompt)}")
        print(f"haiku encode utf8 len prompt: {len(prompt.encode('utf-8'))}")
        
        response = client.messages.create(
            model="anthropic.claude-3-haiku-20240307-v1:0",
            max_tokens=1500,
            temperature=0.5,
            messages=[
                {"role": "user", "content": prompt}
            ]
        )
        
        input_tokens = response.usage.input_tokens
        output_tokens = response.usage.output_tokens
        print(f"haiku each Input Tokens: {input_tokens}, Output Tokens: {output_tokens}")

        input_tokens_result += input_tokens
        output_tokens_result += output_tokens
        
        thread_analysis = response.content[0].text
        
        thread_comment_analysis += f"\nTop Level Comment: {top_comment[:50]}...\n"
        thread_comment_analysis += f"Sentiment: {thread_sentiment}\n"
        thread_comment_analysis += f"Analysis:\n{thread_analysis}\n\n"
        
    print(f"haiku input token: {input_tokens_result}")
    print(f"haiku output token: {output_tokens_result}")
        
    
    # 분석 결과 요약
    summary = f"""비디오 제목: {video_info['Title']}
    비디오 설명: {video_info['Description']}
       
    1. 상위 인기 댓글 (좋아요 순 상위 30개):
    {
        ''.join([f"   - {c['topLevelComment'][:20]}... (좋아요: {c['likeCount']}... {c['ThreadComment'][:200]}) " for c in top_comments[:30]])
    }
    
    2. 주요 댓글 클러스터:
    {
        ''.join([f"   클러스터 {i}: {Counter(cluster_labels)[i]}개 댓글 " for i in range(100)])
    }
    
    3. Thread가 많은 상위 10개 댓글 분석:
    {thread_comment_analysis}
    
    4. 전반적인 분석:
      1) 분석에 근거가 되는 원문을 포함해서 댓글의 전반적인 감정(긍정적, 부정적, 중립적)을 파악해주세요.
      2) 분석에 근거가 되는 원문을 포함해서 토론의 주된 톤(예: 열정적, 비판적, 정보 제공적, 유머러스 등)을 결정해주세요.
      3) 분석에 근거가 되는 원문을 포함해서 댓글 작성자들이 반복적으로 제기하는 질문, 우려사항, 관심사를 강조해주세요.
      4) 분석에 근거가 되는 원문을 포함해서 댓글에서 나타나는 중요한 의견 차이나 논쟁을 파악해주세요.
      5) 분석에 근거가 되는 원문을 포함해서 비디오 제작자에게 제시된 주목할 만한 피드백이나 제안을 확인해주세요.
      6) 분석에 근거가 되는 원문을 포함해서 Thread가 많은 상위 10개 댓글 요약 분석에 대한 내용을 포함해서 별도로 작성해주고 안에 원문은 그대로 유지하세요.

        비디오 설명이 제공하는 컨텍스트와 댓글 작성자들 간의 상호작용을 고려하여 종합적인 분석을 제공해주세요.
        분석에 근거가 되는 원문을 생략없이 최대한 많이 포함해서 작성해주세요.
    """
    
    print(f"large summary prompt: {len(prompt)}")
    print(f"large summary prompt: {len(prompt.encode('utf-8'))}")
    
    response = client.messages.create(
            model="anthropic.claude-3-5-sonnet-20240620-v1:0",
            max_tokens=4000,
            temperature=0.7,
            messages=[
                {"role": "user", "content": summary}
            ]
        )
    
    input_tokens = response.usage.input_tokens
    output_tokens = response.usage.output_tokens
    print(f"large 3.5 sonnet input token: {input_tokens}")
    print(f"large 3.5 sonnet output token: {output_tokens}")

    result = response.content[0].text

    return result
    
# Function to get replies for a specific comment
def get_replies(youtube, parent_id, video_id):
    replies = []
    request = youtube.comments().list(
        part="snippet",
        parentId=parent_id,
        textFormat="plainText",
        maxResults=100
    )
    
    while request:
        response = request.execute()
        replies.extend([{
            'Timestamp': item['snippet']['publishedAt'],
            'Username': item['snippet']['authorDisplayName'],
            'VideoID': video_id,
            'topLevelComment': response['items'][0]['snippet']['textDisplay'],  # Avoid redundant indexing
            'ThreadComment': item['snippet']['textDisplay'],
            'likeCount': item['snippet']['likeCount']
            # 'Date': item['snippet'].get('updatedAt', item['snippet']['publishedAt'])
        } for item in response['items']])

        request = youtube.comments().list_next(request, response)

    return replies


# Function to get all comments (including replies) for a single video
def get_comments_for_video(youtube, video_id):
    all_comments = []
    request = youtube.commentThreads().list(
        part="snippet",
        videoId=video_id,
        textFormat="plainText",
        maxResults=100
    )
    
    while request:
        response = request.execute()
        for item in response['items']:
            top_comment = item['snippet']['topLevelComment']['snippet']
            all_comments.append({
                'Timestamp': top_comment['publishedAt'],
                'Username': top_comment['authorDisplayName'],
                'VideoID': video_id,
                'topLevelComment': top_comment['textDisplay'],
                'ThreadComment': '',
                'likeCount': top_comment['likeCount']
                # 'Date': top_comment.get('updatedAt', top_comment['publishedAt'])
            })

            # Fetch replies if there are any
            if item['snippet']['totalReplyCount'] > 0:
                all_comments.extend(get_replies(youtube, item['id'], video_id))

        request = youtube.commentThreads().list_next(request, response)

    return all_comments


# Function to get video details (title and description)
def get_video_details(youtube, video_id):
    request = youtube.videos().list(
        part='snippet',
        id=video_id
    )
    response = request.execute()

    if response['items']:
        snippet = response['items'][0]['snippet']
        return {
            'VideoID': video_id,
            'Title': snippet['title'],
            'Description': snippet['description']
        }
    return None

# 비디오 정보와 댓글 가져오기
video_details = []
all_comments = []
video_ids = ['fbIaogYiZt0'] #['IenW7-OrBVU', 'fbIaogYiZt0'] #['fbIaogYiZt0'] #['fbIaogYiZt0']   #['IenW7-OrBVU', 'i1PUY_vJxZs']

for video_id in video_ids:
    video_info = get_video_details(youtube, video_id)
    if video_info:
        video_details.append(video_info)
    video_comments = get_comments_for_video(youtube, video_id)
    all_comments.extend(video_comments)

video_df = pd.DataFrame(video_details)
comments_df = pd.DataFrame(all_comments)

# 각 비디오 분석
analysis_results = []
for _, row in video_df.iterrows():
    video_id = row['VideoID']
    title = row['Title']
    
    if len(comments_df[comments_df['VideoID'] == video_id ]) > 1000:
        analysis_type = 'large'
        analysis = analyze_video_content_large(video_id, video_df, comments_df)
    else:
        analysis_type = 'small'
        analysis = analyze_video_content_small(video_id, video_df, comments_df)
    
    analysis_results.append({
        'VideoID': video_id,
        'Title': title,
        'Analysis-type': analysis_type,
        'Analysis': analysis
    })

analysis_df = pd.DataFrame(analysis_results)
print("Video Analysis DataFrame:")
print(analysis_df)

# Result save as CSV
video_ids_string = "_".join(video_ids)
filename = f'youtube_analysis_{video_ids_string}.csv'
analysis_df.to_csv(filename, index=False)

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/nltk/metrics/association.py:26: UserWarning: A NumPy version >=1.23.5 and <2.3.0 is required for this version of SciPy (detected version 1.22.4)
  from scipy.stats import fisher_exact


small prompt: 2790
small prompt: 4766
small 3.5 sonnet Input Tokens: 1921, Output Tokens: 1181
Video Analysis DataFrame:
       VideoID                                       Title Analysis-type  \
0  fbIaogYiZt0  🌊현시점 가장 힙한 클럽 캐비😎 | #shorts #캐리비안베이 #에버랜드론         small   

                                            Analysis  
0  이 YouTube 비디오 콘텐츠 분석 결과는 다음과 같습니다:\n\n1. 주요 주제...  


In [7]:
analysis_df

,VideoID,Title,Analysis-type,Analysis
0,fbIaogYiZt0,🌊현시점 가장 힙한 클럽 캐비😎 | #shorts #캐리비안베이 #에버랜드론,small,이 YouTube 비디오 콘텐츠 분석 결과는 다음과 같습니다:\n\n1. 주요 주제...
